Notebook to debug the MCS + NLLS approach.

In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib qt

In [2]:
import os
import sys
from pathlib import Path

In [3]:
src_dir = str(Path(os.getcwd()).parent / "src")
sys.path.insert(1, src_dir)
os.environ["PYTHONPATH"] = src_dir

In [4]:
import data_loader
import calculate_concentr_diffs
from concentr_diffs_pathlength import *
from plotting import *
from mbll_functions import *
from utils import *
from mcs_function import *
from pathlib import Path
import config
import pickle
from ray import tune
import pmcx

import matplotlib.pyplot as plt

In [5]:
config.data_dir

PosixPath('/media/leon/T7/IDP')

In [6]:
config.spectra_dir

PosixPath('/media/leon/Storage/Studium/Informatik/Master/3.Semester/IDP/nonlinear-fit/dataset/UCL-NIR-Spectra/spectra')

In [7]:
mu_a_matrices = []
mu_a_matrices_static = []
mu_a_matrices_old = []

In [ ]:
loader = data_loader.DataLoaderHELICOID(
    config.dataset_dir,
    wavelength_left_cut=520,
    wavelength_right_cut=850,
    num_wavelengths=20
)

delta_A_gt_img = loader.get_attenuation_change("020-01")
A_gt_img = loader.get_attenuation("020-01")

mu_a_matrix_old = loader.absorption_coefs_old(
    use_diff_oxycco=False,
    use_water_and_fat=True
)

mu_a_matrices_old.append(mu_a_matrix_old)

mu_a_matrix = loader.absorption_coefs(
    use_diff_oxycco=False,
    use_water_and_fat=True
)

mu_a_matrices.append(mu_a_matrix)

mu_a_matrix_static = data_loader.DataLoader.absorption_coefs(
    loader.wavelengths,
    use_diff_oxycco=False,
    use_water_and_fat=True
)

mu_a_matrices_static.append(mu_a_matrix_static)

print(np.allclose(mu_a_matrix, mu_a_matrix_static))
print(np.max(np.abs(mu_a_matrix - mu_a_matrix_static)))

False
101.71565016290911


In [ ]:
print([np.max(m - mu_a_matrices[0]) for m in mu_a_matrices])
print([np.max(m - mu_a_matrices_static[0]) for m in mu_a_matrices_static])
print([np.max(m - mu_a_matrices_old[0]) for m in mu_a_matrices_old])

[0.0, 101.71565016290911]
[0.0, 2.14321574911e-312]


In [ ]:
plot_spectra(mu_a_matrix_old[:, :4].T, loader.wavelengths, ["HbO2", "Hbb", "oxCCO", "redCCO", "Water", "Fat"])

In [ ]:
plot_spectra(mu_a_matrix[:, :4].T, loader.wavelengths, ["HbO2", "Hbb", "oxCCO", "redCCO", "Water", "Fat"])

In [ ]:
loader.wavelengths

In [ ]:
mcs_obj1 = SimulationAttenuation(config.mcs_func_path.parent / "function_data1.npz")
mcs_obj2 = SimulationAttenuation(config.mcs_func_path.parent / "function_data2.npz")

Create spectrum with MCS and compare to MCS-func

In [ ]:
g_combined = 0.8675
refractive_index_combined = 1.38
vol = np.ones((50, 50, 100))
prop = np.array([[0, 0, 1, 1], [0, 0, g_combined, refractive_index_combined]])
unitinmm = 1

cfg = {
    "nphoton": 1e8, # keep nphotons relatively low, to limit memory of stored data
    "maxdetphoton": 1e8,
    "unitinmm": unitinmm,
    "vol": vol,
    "tstart":0,
    "tend":1e-8,
    "tstep":1e-8,
    "autopilot": 1,
    "gpuid": config.gpuid,
    "prop":prop,
    "bc": "ccrcca001000", # mark z=0 plane as detector
    "srcdir": [0,0,1],
    "srctype": "planar",
    "srcpos": [0, 0, 0],
    "srcparam1": [vol.shape[0], 0, 0, 0], # 3D position of vertex, fourth coordinate is irrelevant
    "srcparam2": [0, vol.shape[1], 0, 0],
    "issrcfrom0": 1,
    "savedetflag": "dps", # detector id, path length, scatter count
    "flog": config.mcs_func_path / "log.txt",
}

In [ ]:
mu_a_vals = loader.mu_a_func_gray_matter(loader.wavelengths)
mu_s_vals = loader.mu_s_red_func_gray_matter(loader.wavelengths)

In [ ]:
A_mcs_generated = np.empty((loader.wavelengths.shape[0], ))

for i, wl in enumerate(loader.wavelengths):
    cfg["prop"][1, 0] = mu_a_vals[i] / 10
    cfg["prop"][1, 1] = mu_s_vals[i] / 10
    cfg["prop"][1, 2] = g_combined
    cfg["prop"][1, 3] = refractive_index_combined
    print(f"Iteration {i}/{loader.wavelengths.shape[0]}")
    res = pmcx.mcxlab(cfg)
    weights = pmcx.utils.detweight(res["detp"])
    attenuation = -np.log(np.sum(weights)/cfg["nphoton"])
    A_mcs_generated[i] = attenuation

In [ ]:
A_mcs_func_computed1 = mcs_obj1.A_concentrations(
    loader.wavelengths, 
    mu_a_matrix, 
    loader.params_ref_gray_matter[:6],
    *loader.params_ref_gray_matter[-2:]
)

A_mcs_func_computed2 = mcs_obj2.A_concentrations(
    loader.wavelengths, 
    mu_a_matrix, 
    loader.params_ref_gray_matter[:6],
    *loader.params_ref_gray_matter[-2:]
)

In [ ]:
plot_spectra(
    ( A_mcs_func_computed1, A_mcs_func_computed2),
    loader.wavelengths,
    labels=["Generated", "Computed1", "Computed2"]
)